In [9]:
import cv2
import numpy as np
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk

# Tamanho máximo da imagem
MAX_WIDTH = 400
MAX_HEIGHT = 300

# Variáveis globais
img_original = None
img_atual = None
historico_filtros = []  # Lista para armazenar o histórico de filtros aplicados
historico_imagens = []  # Lista para armazenar as versões da imagem após cada filtro

def carregar_imagem():
    """Carrega uma imagem e exibe na interface."""
    global img_original, img_atual, historico_filtros, historico_imagens
    caminho_imagem = filedialog.askopenfilename()
    if caminho_imagem:
        img_original = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
        img_original = redimensionar_imagem(img_original, MAX_WIDTH, MAX_HEIGHT)
        img_atual = img_original.copy()
        historico_filtros.clear()  # Limpa o histórico de filtros
        historico_imagens.clear()  # Limpa o histórico de imagens
        historico_imagens.append(img_atual)  # Adiciona a imagem original no histórico
        atualizar_previa()

def redimensionar_imagem(img, max_width, max_height):
    """Redimensiona a imagem para caber dentro de dimensões máximas."""
    altura, largura = img.shape
    escala = min(max_width / largura, max_height / altura)
    nova_largura, nova_altura = int(largura * escala), int(altura * escala)
    return cv2.resize(img, (nova_largura, nova_altura))

def atualizar_previa():
    """Atualiza a exibição da imagem original."""
    if img_atual is not None:
        img_exibicao = ImageTk.PhotoImage(Image.fromarray(img_atual))
        painel_imagem_original.config(image=img_exibicao)
        painel_imagem_original.image = img_exibicao

def aplicar_filtro():
    """Aplica o filtro selecionado e exibe o resultado."""
    global img_atual, historico_filtros, historico_imagens
    if img_atual is None:
        return

    filtro = combo_filtros.get()
    kernel_size = slider_kernel.get()
    intensidade = slider_intensidade.get()

    img_resultante = img_atual.copy()

    if filtro == "Passa-Baixa (Gaussiano)":
        img_resultante = cv2.GaussianBlur(img_resultante, (kernel_size, kernel_size), 0)
    elif filtro == "Passa-Alta":
        img_blur = cv2.GaussianBlur(img_resultante, (kernel_size, kernel_size), 0)
        img_resultante = cv2.subtract(img_resultante, img_blur)
    elif filtro == "Erosão":
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
        img_resultante = cv2.erode(img_resultante, kernel, iterations=1)
    elif filtro == "Dilatação":
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
        img_resultante = cv2.dilate(img_resultante, kernel, iterations=1)
    elif filtro == "Limiarização":
        _, img_resultante = cv2.threshold(img_resultante, intensidade, 255, cv2.THRESH_BINARY)

    # Atualiza o histórico de filtros e imagens
    historico_filtros.append((filtro, kernel_size, intensidade))
    historico_imagens.append(img_resultante)

    img_atual = img_resultante
    atualizar_previa_filtrada()
    atualizar_historico_filtros()

def atualizar_previa_filtrada():
    """Atualiza a exibição da imagem filtrada."""
    if img_atual is not None:
        img_exibicao_filtrada = ImageTk.PhotoImage(Image.fromarray(img_atual))
        painel_imagem_filtrada.config(image=img_exibicao_filtrada)
        painel_imagem_filtrada.image = img_exibicao_filtrada

def resetar_filtros():
    """Reseta todos os filtros e volta para a imagem original."""
    global img_atual, historico_filtros, historico_imagens
    if img_original is not None:
        img_atual = img_original.copy()
        historico_filtros.clear()
        historico_imagens.clear()
        historico_imagens.append(img_atual)  # Adiciona a imagem original novamente
        atualizar_previa()
        atualizar_previa_filtrada()
        atualizar_historico_filtros()

def atualizar_historico_filtros():
    """Atualiza a exibição do histórico de filtros aplicados."""
    lista_filtros.delete(0, tk.END)
    for filtro, kernel_size, intensidade in historico_filtros:
        lista_filtros.insert(tk.END, f"{filtro} (Kernel: {kernel_size}, Intensidade: {intensidade})")

def desfazer_filtro():
    """Desfaz a última alteração e volta à versão anterior da imagem."""
    global img_atual
    if len(historico_imagens) > 1:
        # Remove a última imagem aplicada e volta à versão anterior
        historico_imagens.pop()
        img_atual = historico_imagens[-1]
        atualizar_previa_filtrada()
        historico_filtros.pop()
        atualizar_historico_filtros()

def criar_interface():
    """Configura a interface gráfica."""
    global painel_imagem_original, painel_imagem_filtrada, combo_filtros, slider_kernel, slider_intensidade, lista_filtros

    janela = tk.Tk()
    janela.title("Editor de Fotos - Filtros com Histórico de Alterações")

    # Painéis de imagem
    painel_imagem_original = tk.Label(janela, text="Imagem Original")
    painel_imagem_original.grid(row=0, column=0, padx=10, pady=10)
    painel_imagem_filtrada = tk.Label(janela, text="Imagem Filtrada")
    painel_imagem_filtrada.grid(row=0, column=1, padx=10, pady=10)

    # Controles
    frame_controles = tk.LabelFrame(janela, text="Controles", padx=10, pady=10)
    frame_controles.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

    # Botão carregar imagem
    tk.Button(frame_controles, text="Carregar Imagem", command=carregar_imagem).grid(row=0, column=0, padx=10, pady=5)

    # Botão resetar filtros
    tk.Button(frame_controles, text="Resetar Filtros", command=resetar_filtros).grid(row=0, column=1, padx=10, pady=5)

    # ComboBox filtro
    tk.Label(frame_controles, text="Filtro:").grid(row=1, column=0, sticky="e")
    combo_filtros = ttk.Combobox(
        frame_controles,
        values=["Passa-Baixa (Gaussiano)", "Passa-Alta", "Erosão", "Dilatação", "Limiarização"],
    )
    combo_filtros.grid(row=1, column=1, padx=10, pady=5)
    combo_filtros.current(0)

    # Slider tamanho do kernel
    tk.Label(frame_controles, text="Tamanho do Kernel:").grid(row=2, column=0, sticky="e")
    slider_kernel = tk.Scale(frame_controles, from_=1, to=31, orient="horizontal")
    slider_kernel.grid(row=2, column=1, padx=10, pady=5)
    slider_kernel.set(3)

    # Slider intensidade
    tk.Label(frame_controles, text="Intensidade:").grid(row=3, column=0, sticky="e")
    slider_intensidade = tk.Scale(frame_controles, from_=1, to=255, orient="horizontal")
    slider_intensidade.grid(row=3, column=1, padx=10, pady=5)
    slider_intensidade.set(128)

    # Botão aplicar filtro
    tk.Button(frame_controles, text="Aplicar Filtro", command=aplicar_filtro).grid(row=4, column=0, columnspan=2, pady=10)

    # Botão desfazer
    tk.Button(frame_controles, text="Desfazer", command=desfazer_filtro).grid(row=5, column=0, columnspan=2, pady=5)

    # Histórico de filtros aplicados
    tk.Label(frame_controles, text="Histórico de Filtros").grid(row=6, column=0, columnspan=2)
    lista_filtros = tk.Listbox(frame_controles, height=5, width=50)
    lista_filtros.grid(row=7, column=0, columnspan=2, padx=10, pady=5)

    janela.mainloop()

if __name__ == "__main__":
    criar_interface()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/tmp/ipykernel_54109/2568738991.py", line 58, in aplicar_filtro
    img_resultante = cv2.GaussianBlur(img_resultante, (kernel_size, kernel_size), 0)
cv2.error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/smooth.dispatch.cpp:293: error: (-215:Assertion failed) ksize.width > 0 && ksize.width % 2 == 1 && ksize.height > 0 && ksize.height % 2 == 1 in function 'createGaussianKernels'

